<a href="https://colab.research.google.com/github/dahyunk-snu/splat/blob/main/XsenViewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/dahyunk-snu/splat
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -O /content/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

Cloning into 'splat'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 169 (delta 80), reused 59 (delta 59), pack-reused 71 (from 2)
Receiving objects: 100% (169/169), 6.29 MiB | 12.84 MiB/s, done.
Resolving deltas: 100% (95/95), done.
--2025-12-23 16:21:02--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb [following]
--2025-12-23 16:21:02--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets

In [3]:
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['cloudflared', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(7860, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)
%cd /content/splat
!python -m http.server 7860

<>:12: SyntaxWarning: invalid escape sequence '\/'
<>:12: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipython-input-969851764.py:12: SyntaxWarning: invalid escape sequence '\/'
  tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")


https://published-trans-try-newman.trycloudflare.com
/content/splat
Serving HTTP on 0.0.0.0 port 7860 (http://0.0.0.0:7860/) ...
127.0.0.1 - - [23/Dec/2025 16:21:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2025 16:21:38] "GET /main.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2025 16:21:38] "GET /output.splat HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2025 16:21:39] code 404, message File not found
127.0.0.1 - - [23/Dec/2025 16:21:39] "GET /favicon.ico HTTP/1.1" 404 -
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 55396)
Traceback (most recent call last):
  File "/usr/lib/python3.12/socketserver.py", line 697, in process_request_thread
    self.finish_request(request, client_address)
  File "/usr/lib/python3.12/http/server.py", line 1311, in finish_request
    self.RequestHandlerClass(request, client_address, self,
  File "/usr/lib/python3.12/http/server.py", line 672, in __init__
    super().__init__(*args, **kwargs)
  File "/us